# Process Rosbag and plot

In [ ]:
# Auto Reload Modules
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
from scipy.signal import lfilter

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rc, font_manager, transforms
from matplotlib.ticker import FormatStrFormatter

from bag_utils import extract_traj
from plot_utils import plot_map_2d, plot_map_3d, plot_car_frames

np.random.seed(0)

## Load bag and extract data

In [ ]:
bag_files = glob.glob("./bag_data/*.bag")
print("Found bags:", bag_files)

In [ ]:
bag_num = 0

FSM_naive, EV_naive, TV_naive = extract_traj(bag_files[bag_num])

In [ ]:
bag_num = 1
FSM_strat, EV_strat, TV_strat = extract_traj(bag_files[bag_num])

## Visualization

In [ ]:
# Generate occupancy randomly
occupancy = np.random.rand(2, 19)
occupancy[occupancy <= 0.5] = 0
occupancy[occupancy > 0.5] = 1
# Set the destination of target vehicle to be empty
occupancy[0, 7] = 0

# Plot the figure in an interactive window
%matplotlib qt

rc('font', **{'family': "Times New Roman"})

### Plot 2D trajectoies

In [ ]:
fig_name = "ParkingLot2D"
fig = plt.figure(fig_name, figsize=(13, 10))

x_boundary = [-0.1, 6.1]
y_boundary = [-1.5, 1.5]

# Vehicle Dimension
W = 0.265
L = 0.52

# Car Icon
blue_car = mpimg.imread('car_icons_blue.png')
green_car = mpimg.imread('car_icons_green.png')

# ======================
# Naive Plot
ax1 = plt.subplot(2, 1, 1)

plot_map_2d(ax1, occupancy=occupancy)

ax1.plot(TV_naive.x, TV_naive.y, linewidth=3, color='g', label="TV", zorder=5)
ax1.plot(EV_naive.x, EV_naive.y, linewidth=3, color='b', label="EV", zorder=5)

ax1.legend(loc=(0.2, 0.65), prop={'size': 10})

ax1.axis("equal")
ax1.set_xlim(left=x_boundary[0], right=x_boundary[1])
ax1.set_ylim(bottom=y_boundary[0], top=y_boundary[1])
ax1.set_title("Direct Method", pad=1)

# The idxs of trajectory sequence to plot
pos_idxs = [600, 700, 800, 900, 1000]

# The list of EV poses to plot
EV_pos_list = [np.array([EV_naive.x[idx], EV_naive.y[idx], EV_naive.heading[idx]]) for idx in pos_idxs]

# Plot EV along the trajectory
plot_car_frames(ax1, EV_pos_list, blue_car, plot_shape=[W, L])

# The corresponding time stamp
EV_pos_times = [EV_naive.state_t[i] for i in pos_idxs]

# Find the corresponding TV poses at similar time steps
TV_pos_list = []
for time in EV_pos_times:
    idx = np.argmin( np.abs( np.array(TV_naive.state_t) - time ) )
    TV_pos_list.append( np.array([TV_naive.x[idx], TV_naive.y[idx], TV_naive.heading[idx]]) )

# Plot TV
plot_car_frames(ax1, TV_pos_list, green_car, plot_shape=[W, L])

ax1.set_axis_off()

# ======================
# Strat Plot
ax2 = plt.subplot(2, 1, 2)

plot_map_2d(ax2, occupancy=occupancy)

ax2.plot(TV_strat.x, TV_strat.y, linewidth=3, color='g', label="TV", zorder=5)
ax2.plot(EV_strat.x, EV_strat.y, linewidth=3, color='b', label="EV", zorder=5)

ax2.legend(loc=(0.2, 0.65), prop={'size': 10})

ax2.axis("equal")
ax2.set_xlim(left=x_boundary[0], right=x_boundary[1])
ax2.set_ylim(bottom=y_boundary[0], top=y_boundary[1])
ax2.set_title("Proposed Method", pad=1)

# The idxs of trajectory sequence to plot
pos_idxs = [700, 800, 900, 1000, 1100]

# The list of EV poses to plot
EV_pos_list = [np.array([EV_strat.x[idx], EV_strat.y[idx], EV_strat.heading[idx]]) for idx in pos_idxs]

# Plot EV along the trajectory
plot_car_frames(ax2, EV_pos_list, blue_car, plot_shape=[W, L])

# The corresponding time stamp
EV_pos_times = [EV_strat.state_t[i] for i in pos_idxs]

# Find the corresponding TV poses at similar time steps
TV_pos_list = []
for time in EV_pos_times:
    idx = np.argmin( np.abs( np.array(TV_strat.state_t) - time ) )
    TV_pos_list.append( np.array([TV_strat.x[idx], TV_strat.y[idx], TV_strat.heading[idx]]) )

# Plot TV
plot_car_frames(ax2, TV_pos_list, green_car, plot_shape=[W, L])

ax2.set_axis_off()

fig.tight_layout()

### Plot 3D trajectories (+Time)

In [ ]:
fig_name = "ParkingLot3D"

fig = plt.figure(fig_name, figsize=(15, 15), dpi=200)

# plot_map_3d(fig_name, occupancy=occupancy)
plot_map_3d(fig_name, height=EV.state_t[-1]-EV.state_t[0], occupancy=occupancy)

ax = plt.gca()

ax.plot3D(TV.x, TV.y, TV.state_t, linewidth=4, color='g', label="TV")
ax.plot3D(EV.x, EV.y, EV.state_t, linewidth=4, color='b', label="EV")

ax.set_yticks(ticks=[-1, 0, 1])
ax.set_zticks(ticks=[0, 5, 10, 15, 20])
# ax.tick_params(labelsize=10)

ax.set_xlabel("X (m)", labelpad=10, fontdict={"fontname": "Times New Roman", "fontsize": 15, "fontweight": "bold"})
ax.set_ylabel("Y (m)", fontdict={"fontname": "Times New Roman", "fontsize": 15, "fontweight": "bold"})
ax.set_zlabel("Time (s)", fontdict={"fontname": "Times New Roman", "fontsize": 15, "fontweight": "bold"})

font = font_manager.FontProperties(family="Times New Roman")

ax.legend(loc=(0.85,0.7), prop=font)

ax.set_box_aspect([2.5,1,1])

fig.tight_layout()

### Plot States and Inputs

In [ ]:
fig_name = "States_Inputs"

fig = plt.figure(fig_name, figsize=(12, 7))

# ================= Speed Subplot
ax_speed = plt.subplot(3, 1, 1)

# Filtering
n = 20
b = [1.0 / n] * n
a = 1

naive_v_filter = lfilter(b, a, EV_naive.v)
strat_v_filter = lfilter(b, a, EV_strat.v)

t_naive = np.array(EV_naive.state_t) - EV_naive.state_t0
ax_speed.plot(t_naive, naive_v_filter, linewidth=5, color='C4', linestyle='--', label="BL")

t_strat = np.array(EV_strat.state_t) - EV_strat.state_t0
ax_speed.plot(t_strat, strat_v_filter, linewidth=5, color='C1', label="SG")

ax_speed.set_ylabel("Speed\n(m/s)", fontsize=25, fontweight="bold")
ax_speed.set_xlim(left=0, right=20)
ax_speed.set_ylim(bottom=-0.6, top=0.6)

ax_speed.set_yticks([-0.5, -0.25, 0, 0.25, 0.5])
ax_speed.tick_params(axis='y', labelsize=20)
ax_speed.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax_speed.tick_params(axis='x', bottom=False, labelbottom=False)

# ================== Input subplot

# Delta
ax_delta = plt.subplot(3, 1, 2)

t_naive = np.array(EV_naive.input_t) - EV_naive.input_t0
ax_delta.plot(t_naive, EV_naive.delta, linewidth=5, color='C4', linestyle='--', label="BL")

t_strat = np.array(EV_strat.input_t) - EV_strat.input_t0
ax_delta.plot(t_strat, EV_strat.delta, linewidth=5, color='C1', label="SG")

ax_delta.set_ylabel("Steering\n(rad)", fontsize=25, fontweight="bold")
ax_delta.set_xlim(left=0, right=20)
ax_delta.set_ylim(bottom=-0.6, top=0.6)

ax_delta.set_yticks([-0.5, -0.25, 0, 0.25, 0.5])
ax_delta.tick_params(axis='y', labelsize=20)
ax_delta.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax_delta.tick_params(axis='x', bottom=False, labelbottom=False)

# Acc
ax_acc = plt.subplot(3, 1, 3)

ax_acc.plot(t_naive, EV_naive.a, linewidth=5, color='C4', linestyle='--', label="BL")

ax_acc.plot(t_strat, EV_strat.a, linewidth=5, color='C1', label="SG")

ax_acc.set_xlabel("Time (s)", fontsize=25, fontweight="bold")
ax_acc.set_ylabel("Accel\n(m/s$^2$)", fontsize=25, fontweight="bold")

ax_acc.set_xlim(left=0, right=20)
ax_acc.set_ylim(bottom=-1.2, top=1.2)


ax_acc.set_yticks([-1, -0.5, 0, 0.5, 1])
ax_acc.tick_params(axis='y', labelsize=20)
ax_acc.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax_acc.tick_params(axis='x', labelsize=20)

plt.subplots_adjust(hspace=0.1)
plt.tight_layout()


### Plot Score and FSM

In [ ]:
fig_name = "Score_FSM"

fig = plt.figure(fig_name, figsize=(12, 6))

# ================= Score Subplot
ax_score = plt.subplot(2, 1, 1)

strat_t = np.array(FSM_strat.t) - FSM_strat.t0
ax_score.plot(strat_t, FSM_strat.score_l, linewidth=5, color='C3', linestyle='solid', label="Go Left")
ax_score.plot(strat_t, FSM_strat.score_r, linewidth=5, color='C5', linestyle='dashed', label="Go Right")
ax_score.plot(strat_t, FSM_strat.score_y, linewidth=5, color='C6', linestyle='dashdot', label="Yield")

ax_score.set_xlim([0, 15])

ax_score.set_ylabel("Confidence", labelpad=50, fontsize=25, fontweight="bold")

ax_score.tick_params(axis='y', labelsize=20)

ax_score.tick_params(axis='x', bottom=False, labelbottom=False)

ax_score.grid(axis='y')

ax_score.legend(fontsize=20)

# ================== Policy subplot

ax_policy = plt.subplot(2, 1, 2)

naive_t = np.array(FSM_naive.t) - FSM_naive.t0

ax_policy.plot(naive_t, FSM_naive.state_idxs, linewidth=5, color='C4', linestyle='--', label="BL")

ax_policy.plot(strat_t, FSM_strat.state_idxs, linewidth=5, color='C1', label="SG")

ax_policy.set_xlabel("Time (s)", fontsize=25, fontweight="bold")
ax_policy.set_ylabel("Policy", fontsize=25, fontweight="bold")

ax_policy.set_xlim([0, 20])
ax_policy.set_ylim([-0.5, 2.5])

ax_policy.set_yticks([0, 1, 2])

FSM_strat.state_names = ["*-OBCA", "Safety", "E-Brake"]

ax_policy.set_yticklabels(FSM_strat.state_names)
ax_policy.tick_params(axis='y', labelsize=20)

ax_policy.tick_params(axis='x', labelsize=20)

ax_policy.grid(axis='y')

plt.subplots_adjust(hspace=0.01)
plt.tight_layout()
